In [1]:
# This is the past implementation approach, please ignore this.

from selenium import webdriver
import math
import pickle
import uuid
import copy
from functools import reduce

from PIL import Image, ImageDraw
import cv2 as cv

In [2]:
prefix = "file:///Users/chenwang/Documents/Manuscript/Entropy_Aesthetic/Aesthetic/source/"
url1 = "http://web.ics.purdue.edu/~gchopra/class/public/pages/webdesign/05_simple.html"
#title = "52nlp.cn"
#title = "bootcdn.cn"
title = "web.ics.purdue.edu"
url = prefix + title + "/" + title + ".html"

windowWidth = 1920
windowHeight = 900

In [3]:
WebpageList = [
#'999*ted.com',
'0*theboneandjointcenter.com',
'1*mozilla.org',
'2*two-n.com',
'3*segmentfault.com',
'4*makeitok.org',
'5*disney.co.jp',
'6*wikipedia.org',
'7*news.yahoo.co.jp',
'8*huxiu.com',
'9*fangdd.com',
'10*cheshi.com',
'11*humblebundle.com',
'12*theatlantic.com',
'13*superprof.fr',
'14*microsoft.com',
'15*opera.com',
'16*labinthewild.org',
'17*nounplus.net',
'18*richyli.com',
'19*pxtoem.com', 
'20*goofy.photo',
'21*javadrive.jp',
'22*jcodecraeer.com',
'23*hdpfans.com',
'24*jiqimao.tv',
'25*clamav.net',
'26*bootcdn.cn',
'27*runoob.com',
'28*tensorfly.cn',
'29*journaldugeek.com', 
'30*matetranslate.com',
'31*kameisyouten.ocnk.net',
'32*gingerweb.jp',
'33*cp.pocky.jp',
'34*aladdin-aic.com',
'35*ho-ginza.net',
'36*pandayori.com',
'37*sekimoto.dental',
'38*kokage-m.com',
'39*coming-saji.com',
'40*bluewood.bitter.jp',
'41*steakland.jp',
'42*showroomprive.com',
'43*imas-cg.net',
'44*filetender.com',
'45*hexo.io',
'46*yinwang.org',
'47*blog.yitianshijie.net',
'48*yatani.jp',
'49*qiita.com',
'50*52nlp.cn',
'51*guidetojapanese.org',
'52*olderadults.mobi',
'53*blog.whatsapp.com',
'54*stratechery.com',
'55*lomake.fi',
'56*theclinic.cl',
'57*bgmaimuna.com',
'58*0dt.net',
'59*web.ics.purdue.edu',
'60*canon-foundation.jp',
'61*docs.opencv.org',
'62*interaction-design.org',
'63*jlpt.jp',
'64*blog.sciencenet.cn',
'65*pantone.com',
'66*sdtech.co.jp',
'67*news.livedoor.com',
'68*gmo.jp',
'69*tokai-tv.com', 
'70*life-is-tech.com',
'71*bloomberg.co.jp',
'72*cerezo.jp',
'73*tech.nikkeibp.co.jp',
'74*jp.techcrunch.com',
'75*capcom.co.jp',
'76*kenkun-jinja.org',
'77*sankei.com',
'78*tech-jp.co.jp',
'79*tech-camp.in',
'80*hasegawa-heart.com',
'81*techplay.jp',
'82*ubejinja.or.jp',
'83*19lou.com',
'84*jiankang.com',
'85*cps.com.cn',
'86*hea.cn',
'87*jfc.or.jp',
'88*trafst.jp',
'89*infoq.com',
'90*secure.j-bus.co.jp',
'91*jcr.incites.thomsonreuters.com',
'92*dna.fr',
'93*macg.co',
'94*cvpr2018.thecvf.com',
'95*chi2019.acm.org',
'96*swellnet.com',
'97*klex.ru',
'98*kudago.com',
'99*theborneopost.com'
]

In [4]:
def imageProcessing(frame):
    res={}
    gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    ret2, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=2)
    # sure background area
    sure_bg = cv.dilate(opening, kernel, iterations=3)
    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
    ret, sure_fg = cv.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg, sure_fg)
    # Marker labelling
    ret, markers = cv.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0
    markers = cv.watershed(frame, markers)
    frame[markers == -1] = [255, 0, 0]
    rrres = []
    sum=0
    for num in markers:
        for key in num:
            sum = sum + 1
            if res.__contains__(key):
                res[key] = res[key] + 1

            else:
                res.__setitem__(key, 1)

    for key in res.keys():
        rrres.append(int(res[key]/sum*100) )
    return rrres

In [5]:
def isBackground(node):
    if node.value_of_css_property('background-image') == 'none' or node.value_of_css_property('background-color') == 'rgba(255, 255, 255, 1)':
        pass
    return True

In [6]:
def findParent(node):
    return node.find_element_by_xpath('./parent::*')

In [7]:
def findChildren(node):
    return node.find_elements_by_xpath('./child::*') 

In [8]:
def hsvColorMetric(a, b):
    h1, s1, v1 = a.HSV()
    h2, s2, v2 = b.HSV()
    n = math.sqrt((h1 - h2)**2 + (s1 - s2)**2 + (v1 - v2)**2)
    return (0.5)**n

In [9]:
def alignMetric(a, b):
    if a.top == b.top or a.left == b.left:
        if a.width == b.width or a.height == b.height:
            n = 0
        else:
            n = 1
    else:
        n = 2
    return (0.5)**n 

In [10]:
def spacingMetric(a, b):
    return (0.5)**n

In [11]:
def semanticMetric(a, b):
    try:
        if a.parent == b.parent:
            n = 0
        else:
            if a.parent == findParent(b.parent) or findParent(a.parent) == b.parent:
                n = 1
            else:
                if findParent(findParent(a.parent)) == b.parent or a.parent == findParent(findParent(b.parent)):
                    n = 2
                else:
                    n = 8
    except Exception as e:
        print(e)
        n = 1
    return (0.5)**n

In [228]:
def metricPolynomial(a, b):
    metricList = [ 
        alignMetric(a,b), 
        semanticMetric(a,b)]
    # spacingMetric(a,b), 
    # hsvColorMetric(a,b),
    coefficient = 1/len(metricList)
    z = reduce(lambda x,y: x+y, map(lambda v: coefficient*v, metricList))
    return z

In [229]:
def verifyPageHeight(node):
    heightPixel = 900
    if node.top <= heightPixel and node.display != 'none' and node.width not in [0, 1]:
        return True
    else:
        return False

In [230]:
class Node():
    def __init__(self, node):
        self.uuid = uuid.uuid1()
        self.me = node
        self.parent = findParent(node)
        self.tag = node.tag_name
        self.text = node.text
        self.font_size = node.value_of_css_property('font-size')
        self.float = node.value_of_css_property("float")
        #self.top = float(node.get_attribute('offsetTop'))
        #self.left = float(node.get_attribute('offsetLeft'))
        #self.width, self.height = self.textRealPosition(node.value_of_css_property('width'), node.value_of_css_property('height'))
        self.top = node.rect['y']
        self.left = node.rect['x']
        self.width = node.rect['width']
        self.height = node.rect['height']
        self.background_image = node.value_of_css_property('background-image')
        self.background_color = node.value_of_css_property('background-color')
        self.display = node.value_of_css_property('display')
        
        self.p = 0
        self.mark = 0
        
    def textRealPosition(self, width, height):
        try:
            if self.tag in textType or width == 'auto' or height == 'auto':
                widthReal = float(self.font_size.rstrip("px")) * len(self.text) * 0.5
                heightReal = float(self.font_size.rstrip("px"))
            elif width[-1] == '%' or height[-1] == '%':
                widthReal = float(Node(self.parent).width) * width
                heightReal = float(Node(self.parent).height) * height
            else:
                widthReal = float(width.rstrip("px"))
                heightReal = float(height.rstrip("px"))
        except Exception as e:
            print("Node Exception:" + str(e))
        return widthReal, heightReal
    
    '''def originProcessing(self, tag, others, parent):
        if self.float == "left" or 'none':
            left = float(left)
        elif self.float == "right":
            left = float(self.parent.get_attribute('offsetLeft')) + float(self.parent.value_of_css_property('width')) - float(self.width)
        else:
            left = float(left)
        return left'''

In [231]:
def getNodePosition(node):
    print("***", node.left, node.top, node.width, node.height)
    
    if node.top + node.height > windowHeight:
        node.height = windowHeight - node.top
    
    xy = [(node.left, node.top),
          (node.left + node.width, node.top),
          (node.left + node.width, node.top + node.height),
          (node.left, node.top + node.height),
          (node.left, node.top)
         ]
    print(node.left, node.top, node.width, node.height)
    return xy

In [232]:
def getLoop(node):
    n = node.find_elements_by_xpath('./ancestor::*')
    loop = len(n) - 2
    return loop

In [233]:
def verifyPC(children):
    '''if parentNode.node.value_of_css_property('background-image'):
        top = children.get_attribute('offsetTop')
        left = children.get_attribute('offsetLeft')
        width = children.value_of_css_property('width')
        height = children.value_of_css_property('height') '''
    node = Node(children[0].parent)
    return node

In [234]:
def entropyComputing(childStack):
    E = 0
    for i in childStack:
        probability = (i.width * i.height) / (1920*905)
        print(probability)
        if probability == 0:
            continue
        E += probability * math.log(probability,2)
    return -E

In [235]:
def combineElement(a, b):
    print("---" + a.tag)
    print("---" + b.tag)
    print(a)
    print(b)
    c = Node(b.me)
    # c = copy.deepcopy(b)
    if a.left <= b.left:
        c.left = a.left
    else:
        c.left = b.left
    if a.top <= b.top:
        c.top = a.top
    else:
        c.top = b.top
    if a.left + a.width <= b.left + b.width:
        c.width = b.left + b.width - c.left
    else:
        c.width = a.left + a.width - c.left
    if a.top + a.height <= b.top + b.height:
        c.height = b.top + b.height - c.top
    else:
        c.height = a.top + a.height - c.top
    c.me = [a.me, b.me]
    c.parent = findParent(a.parent)
    c.p = metricPolynomial(a,b)
    print("p="+str(c.p)+ "---" + a.tag + "---" + b.tag)
    return c

In [236]:
p = 0.4
def checkingElements(cacheStack):
    originLength = len(cacheStack)
    cacheStack = list(filter(lambda x: verifyPageHeight(x), cacheStack))
    length = len(cacheStack)
    combineStack = []
    if length == 0:
        return combineStack
    elif length == 1:
        return [verifyPC(cacheStack)]
    else:
        cacheTable = [[0 for x in range(length)] for y in range(length)]
        cacheTable[0] = cacheStack
        # print(cacheTable)
        for i in range(length-1):
            for j in range(1+i, length):
                if cacheTable[i][j] == 0:
                    continue
                else:
                    cacheTable[i+1][j] = combineElement(cacheTable[i][j-1], cacheTable[0][j])
        print(cacheTable)
        a = 0
        while a != length:
            value = 0
            for u in range(length-a):
                if cacheTable[u][a+u].p >= p:
                    value = cacheTable[u][a+u].p
                    px = u
                    py = u + a
            if value != 0:
                combineStack.append(cacheTable[px][py])
                a += (u+1)
            else:
                combineStack.append(cacheTable[0][position])
                a += 1
    return combineStack

In [237]:
# driver = webdriver.Firefox()

driver = webdriver.Chrome()
driver.set_window_size(windowWidth, windowHeight)


#screenshot = Image.open("webpages/" + title + ".png")
#screenshot = Image.open(driver.title + ".png")
#draw = ImageDraw.Draw(screenshot)

textType = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'strong', 'p', 'span', 'i', 'a']
originPath = '/html/body'
path = originPath
generalStack = []
parentStack = []
childStack = []
cacheStack = []

for i in WebpageList[:1]:
    wid, title = i.split('*')
    url = prefix + title + "/" + title + ".html"
    driver.get(url)
    screenshot = Image.open("webpages/" + title + ".png")
    draw = ImageDraw.Draw(screenshot)
    
    element = driver.find_element_by_xpath(path)
    generalStack.append(element)
    
    loopMarker = 3
    
    while generalStack:
        currentElement = generalStack.pop()
        print(currentElement.tag_name)
        if findChildren(currentElement) and getLoop(currentElement) < loopMarker:
            parentStack.append(currentElement)
            for child in findChildren(currentElement)[::-1]:
                generalStack.append(child)
        else:
            node = Node(currentElement)
            childStack.append(node)
            # print(childStack)
            '''
            if node.me == findChildren(parentStack[-1])[-1]:
                for j in childStack:
                    if j.parent == parentStack[-1]:
                        v = childStack.index(j)
                        break
                cacheStack = childStack[v:]
                cacheStack = checkingElements(cacheStack)
                print(cacheStack)
                if cacheStack == []:
                    break
                else:
                    childStack[v:] = []
                    childStack.extend(cacheStack)
                    parentStack.pop()
                    cacheStack = []
            else:
                continue
    '''
    print(childStack)
    childStack = list(filter(lambda x: x.top <= windowHeight, childStack))
    #childStack = addWhiteBlock(childStack)
    for i in childStack:
        xy = getNodePosition(i)
        # print(xy)
        draw.line(xy, fill="#FF0000", width=3)
    E = entropyComputing(childStack)
    print(E)
    screenshot.save(title + "-processed.png")

'''
for i in WebpageList:
    wid, title = i.split('*')
    url = prefix + title + "/" + title + ".html"
    driver.get(url)
    screenshot = Image.open("webpages/" + title + ".png")
    draw = ImageDraw.Draw(screenshot)

    element = driver.find_element_by_xpath(path)
    generalStack.append(element)

    loopMarker = 3

    while generalStack:
        currentElement = generalStack.pop()
        print(currentElement.tag_name)
        if findChildren(currentElement) and getLoop(currentElement) < loopMarker:
            parentStack.append(currentElement)
            for child in findChildren(currentElement)[::-1]:
                generalStack.append(child)
        else:
            node = Node(currentElement)
            childStack.append(node)
            # print(childStack)
            if node.me == findChildren(parentStack[-1])[-1]:
                for j in childStack:
                    if j.parent == parentStack[-1]:
                        v = childStack.index(j)
                        break
                cacheStack = childStack[v:]
                cacheStack = checkingElements(cacheStack)
                print(cacheStack)
                if cacheStack == []:
                    break
                else:
                    childStack[v:] = []
                    childStack.extend(cacheStack)
                    parentStack.pop()
                    cacheStack = []
            else:
                continue

    print(childStack)
    childStack = list(filter(lambda x: x.top <= windowHeight, childStack))
    #childStack = addWhiteBlock(childStack)
    for i in childStack:
        xy = getNodePosition(i)
        # print(xy)
        draw.line(xy, fill="#FF0000", width=3)
    E = entropyComputing(childStack)
    print(E)
    screenshot.save(title + "-processed.png")
    '''

body
p
a
div
div
div
div
div
div
div
header
div
div
div
div
div
div
iframe
style
div
iframe
style
div
div
footer
div


WebDriverException: Message: unknown error: getting size failed to return y
  (Session info: chrome=72.0.3626.96)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.14.2 x86_64)


In [ ]:
screenshot.save(title + "-processed.png")

In [73]:
#driver.find_element_by_xpath('./html/body/div/header/nav/div/ul/li[6]/ul').get_attribute('offsetLeft')
#childStack[8].me[0].text

In [26]:
'''
childStack = [[480.0,72.0,221.0,61.0],
    [0.0,0.0,0.0,11.0],
    [960.0,168.0,1.0,1.0],
    [480.0,169.0,12.0,12.0],
    [544.0,169.0,18.0,12.0],
    [624.0,169.0,12.0,12.0],
    [692.0,169.0,24.0,12.0],
    [784.0,169.0,24.0,12.0],
    [0.0,0.0,24.0,44.0],
    [0.0,44.0,72.0,12.0],
    [943.0,169.0,12.0,12.0],
    [1011.0,169.0,12.0,12.0],
    [480.0,262.0,792.0,22.0],
    [480.0,338.0,624.984,24.0],
    [480.0,386.0,189.0,14.0],
    [516.0,434.0,329.0,38.0],
    [480.0,722.0,644.0,14.0],
    [563.0,841.0,613.0,32.0],
    [480.0,1068.0,105.0,14.0],
    [480.0,1116.0,0.0,14.0]]
'''
import numpy as np

testURL = "test.png"
im = Image.open(testURL)
imarr = np.array(im)
draw = ImageDraw.Draw(im)
imWidth, imHeight = im.size

class SimpleNode():
    def __init__(self, position):
        self.left = position[0]
        self.top = position[1]
        self.width = position[2]
        self.height = position[3]
        
def addWhiteBlock(childStack):
    colorCache = ''
    positionCache = []
    areaThreshold = (3, 3)
    childStackCache = []

    for i in range(imHeight):
        for j in range(imWidth):
            for k in childStack:
                #print(i,j,k)
                if j >= k[0] and j <= k[0]+k[2] and i >= k[1] and i <= k[1]+k[3]:
                    if positionCache != [] and positionCache[2] > areaThreshold[0] and positionCache[3] >areaThreshold[1]:
                        print("in",positionCache)
                        print("---less than")
                        simplenode = SimpleNode(positionCache)
                        childStackCache.append(simplenode)
                        colorCache = ''
                        positionCache = []
                        ##
                        xy = getNodePosition(simplenode)
                        print(xy)
                        draw.line(xy, fill="#FF0000", width=3)
                    break
            if colorCache == '':
                colorCache = im.getpixel((j, i))
                positionCache = [j, i, 0, 0]
                print("new",positionCache)
            else:
                if im.getpixel((j, i)) == colorCache:
                    positionCache[2:] = [j-positionCache[0], i-positionCache[1]]
                    print(positionCache)
                else:
                    if positionCache[2] > areaThreshold[0] and positionCache[3] > areaThreshold[1]:
                        print("less than")
                        simplenode = SimpleNode(positionCache)
                        childStackCache.append(simplenode)
                        ##
                        xy = getNodePosition(simplenode)
                        print("get",xy)
                        draw.line(xy, fill="#FF0000", width=3)
                        ##
                    colorCache = ''
                    positionCache = []                    
                        
    childStack.extend(childStackCache)
    return childStack

addWhiteBlock(childStack)
im.save("p-test.png")
print("end")

FileNotFoundError: [Errno 2] No such file or directory: 'test.png'